### This notebook retrieves guideRNA predictions to create gene knockouts for human gene sequence

In [9]:
#import modules
from pybiomart import Server
import pandas as pd
import urllib.request
# import nest_asyncio
# nest_asyncio.apply()
from selenium import webdriver
import time
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [10]:
#request user input; if none supplied by default it is ENSG00000141510 (TP53)
print('Input Ensembl Gene Id number - for example "ENSG00000141510": ')
gene_ens_id = input()
gene_ens_id = gene_ens_id or 'ENSG00000141510'

Input Ensembl Gene Id number - for example "ENSG00000141510": 
ENSG00000134323


In [11]:
#connect to Biomart Server
server = Server(host='http://www.ensembl.org')

# use #dataset.list_filters() to see available filters for list_filters method
#dataset.list_filters()

#generate dataset (homo sapiens)
dataset = (server.marts['ENSEMBL_MART_ENSEMBL']
                 .datasets['hsapiens_gene_ensembl'])


In [12]:
#parse dataset for gene_id
data_gene = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name'],
              filters={'link_ensembl_gene_id': gene_ens_id})

#if gene_id found and unique, construct Synthego url
if len(data_gene.index) == 1:
    #print(data_gene)
    gene_id = data_gene.iat[0,0]
    gene_symbol = data_gene.iat[0,1]
    url = 'https://design.synthego.com/#/design/results?genome=homo_sapiens_gencode_26_primary&nuclease=cas9&gene_id='+gene_id+'&symbol='+gene_symbol
else:
    raise SystemExit("Stop right there - ambiguous gene name!")
data_gene

,Gene stable ID,Gene name
0,ENSG00000134323,MYCN


In [13]:
# specify the url
urlpage = url
print(urlpage)
# run firefox webdriver from executable path of your choice
driver = webdriver.Firefox(executable_path = 'D:\Alex\geckodriver-v0.31.0-win64\geckodriver.exe')
# get web page
driver.get(urlpage)
#60 seconds wait to have page fully loaded - might need to increase time
time.sleep(60)
results = driver.find_elements("xpath", "//*[@class='ng-binding']")
#print('Number of results', len(results))

https://design.synthego.com/#/design/results?genome=homo_sapiens_gencode_26_primary&nuclease=cas9&gene_id=ENSG00000134323&symbol=MYCN


In [14]:
#Store predictions
gRNAs_synth = []
# loop over results
for result in results:
    #print(result.text)
    gRNAs_synth.append(result.text)

In [15]:
#extract minimal information for predicted gRNAs
gRNAs_synth_min = gRNAs_synth[3:9]

In [16]:
#create pandas dataframe for future manipulation
gRNAs_synth_min_df = pd.DataFrame(gRNAs_synth_min[2:], index =[gene_symbol+'-gRNA1', gene_symbol+'-gRNA2', gene_symbol+'-gRNA3', gene_symbol+'-gRNA4'],
                                             columns =['gRNA sequence'])
#add column(s) with ENSG number/gene name to df
gRNAs_synth_min_df['ENSG ID'] = gene_id
gRNAs_synth_min_df['Gene Name'] = gene_symbol
#save df
gRNAs_synth_min_df.to_csv('gRNAs_synth_min_df.csv') 
#sanity check df
gRNAs_synth_min_df

,gRNA sequence,ENSG ID,Gene Name
MYCN-gRNA1,GACGUGGAGCAGCUCGGCAU,ENSG00000134323,MYCN
MYCN-gRNA2,AUGGUGGACGUGGAGCAGCU,ENSG00000134323,MYCN
MYCN-gRNA3,CAUGCCCGGCAUGGUGGACG,ENSG00000134323,MYCN
MYCN-gRNA4,CUGCUCCACGUCCACCAUGC,ENSG00000134323,MYCN
